In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el dataset (suponiendo que el archivo este disponible en formato CSV)

df_wine = pd.read_csv("C:/Users/sopor/Desktop/Curso Coding Dojo/Machine Learning/Prediccion Vino/WineQT.csv")

# Mostrar las primeras filas 
df_wine.head()

# Obtener informacion basica
df_wine.info()

# Descripcion estadistica de las variables
df_wine.describe()

# Visualizacion de la distribucion de la variable calidad del vino
sns.countplot(x="quality", data=df_wine)
plt.title("Distribución de la Calidad del Vino")
plt.show()

# Revisar valores nulos
df_wine.isnull().sum()

# Visualizacion de posibles outliers mediante boxplots
for col in df_wine.columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df_wine[col])
    plt.title(f'Outliers en {col}')
    plt.show()

# Crear categorias para la variable quality
df_wine["quality_category"] = pd.cut(df_wine["quality"], bins=[0, 4, 6, 10], labels=["Baja", "Media", "Alta"])

# Ver las primeras filas
df_wine.head()


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir datos en variables predictoras (X) y objetivo (y)
X = df_wine.drop(["quality", "quality_category"], axis=1)
y = df_wine["quality_category"]  # O 'quality' si prefieres un enfoque regresivo

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.preprocessing import StandardScaler

# Escalar las caracteristicas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Definir el modelo y los parametros para GridSearch
knn = KNeighborsClassifier()
param_grid = {"n_neighbors": [3, 5, 7, 9, 11]}

# Buscar los mejores parámetros
grid_search_knn = GridSearchCV(knn, param_grid, cv=5)
grid_search_knn.fit(X_train_scaled, y_train)
# Mejor modelo
best_knn = grid_search_knn.best_estimator_

from sklearn.ensemble import RandomForestClassifier

# Definir y entrenar el modelo
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)

from sklearn.linear_model import LogisticRegression

# Definir y entrenar el modelo
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Funcion para evaluar los modelos
def evaluar_modelo(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    print(f"Reporte de Clasificacion para {modelo.__class__.__name__}:\n")
    print(classification_report(y_test, y_pred))
    print("Matriz de Confusion:")
    print(confusion_matrix(y_test, y_pred))
    print("Exactitud:", accuracy_score(y_test, y_pred))

# Evaluar los tres modelos
evaluar_modelo(best_knn, X_test_scaled, y_test)
evaluar_modelo(rf, X_test_scaled, y_test)
evaluar_modelo(lr, X_test_scaled, y_test)

from sklearn.metrics import roc_curve, auc

# Graficar la curva ROC
def curva_roc(modelo, X_test, y_test):
    y_pred_prob = modelo.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob, pos_label='Alta')
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Curva ROC - {modelo.__class__.__name__}')
    plt.legend(loc="lower right")
    plt.show()

# Graficar la curva ROC para el mejor modelo
curva_roc(best_knn, X_test_scaled, y_test)
